<a href="https://colab.research.google.com/github/dbckz/crossing-the-line/blob/master/notebooks/processing_wc_qf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
import pandas as pd
import numpy as np
import ast
import os
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from tqdm import tqdm
from google.colab import drive
import plotly.graph_objects as go

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Set up paths
root_path = "/content/drive/MyDrive/University/Dissertation/world_cup_data"
graph_path = root_path + "/graphs"

day_paths = day_paths = [
        "/021222",
        "/031222",
        "/041222",
        "/051222",
        "/061222",
        "/071222",
        "/081222",
        "/091222",
        "/101222",
        "/111222",
        "/121222",
        "/131222",
        "/141222",
        "/151222",
        "/161222",
        "/171222",
        "/181222",
        "/191222"
    ]

In [5]:
# Load data
threshold = '50'
in_tweets = pd.DataFrame()
hb_guard = pd.DataFrame()
emojis = pd.DataFrame()
persp = pd.DataFrame()

for path in day_paths:
    directory = root_path + path
    tweets_csv = directory + "/tweets.csv"
    persp_csv = directory + "/perspective_processed_tweets.csv"
    emojis_csv = directory + "/emoji.csv"

    print(f"Loading CSVs for directory {path}...")
    in_tweets = pd.concat([in_tweets, 
                           pd.read_csv(tweets_csv,
                                       usecols = [
                                                  'created_at',
                                                  'tweet_id',
                                                  'tweet_text',
                                                  'accounts_mentioned'
                                       ],
                                       dtype = {
                                          # 'created_at':
                                          'tweet_id': np.int64,
                                          'tweet_text': str,
                                          'accounts_mentioned': object
                                       },
                                       parse_dates=['created_at'])])

    persp = pd.concat([persp, pd.read_csv(persp_csv,
                                          usecols = [
                                                      'tweet_id',
                                                      'identity_attack_score',
                                                      'threat_score',
                                                      'severe_toxicity_score',
                                                      'insult_score'
                                                      
                                          ],
                                          dtype = {
                                              'tweet_id': np.int64,
                                              'identity_attack_score': float,
                                              'threat_score': float,
                                              'severe_toxicity_score': float,
                                              'insult_score': float
                                          })])

# Dedup
original_tweets_length = len(in_tweets)
original_persp_length = len(persp)
in_tweets.drop_duplicates(subset=['tweet_id'], inplace=True)
persp.drop_duplicates(subset=['tweet_id'], inplace=True)

print(f"Size of tweets dataframe: {len(in_tweets)}, having dropped {original_tweets_length - len(in_tweets)} duplicate rows")
print(f"Size of perspective dataframe: {len(persp)}, having dropped {original_persp_length - len(persp)} duplicate rows")

Loading CSVs for directory /021222...
Loading CSVs for directory /031222...
Loading CSVs for directory /041222...
Loading CSVs for directory /051222...
Loading CSVs for directory /061222...
Loading CSVs for directory /071222...
Loading CSVs for directory /081222...
Loading CSVs for directory /091222...
Loading CSVs for directory /101222...
Loading CSVs for directory /111222...
Loading CSVs for directory /121222...
Loading CSVs for directory /131222...
Loading CSVs for directory /141222...
Loading CSVs for directory /151222...
Loading CSVs for directory /161222...
Loading CSVs for directory /171222...
Loading CSVs for directory /181222...
Loading CSVs for directory /191222...
Size of tweets dataframe: 168656, having dropped 5 duplicate rows
Size of perspective dataframe: 168656, having dropped 23 duplicate rows


In [6]:
# Up the pandas display limits so printed dataframes aren't so truncated
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Data manipulation

In [8]:
# Join tables + drop old ones!
joined_df = pd.merge(in_tweets, persp, how='outer', on='tweet_id')
del in_tweets
del persp

In [9]:
joined_df = joined_df[(joined_df['created_at'] > '2022-12-09 00:00:00+00:00') & (joined_df['created_at'] < '2022-12-16 00:00:00+00:00')]
len(joined_df)

98678

In [12]:
# Extract England players
list_of_players = ["JPickford1", "kylewalker2", "LukeShaw23", "_DeclanRice", "HarryMaguire93", "JackGrealish",
                    "JHenderson", "HKane", "sterling7", "MarcusRashford", "trippier2", "deanhenderson",
                    "Kalvinphillips", "OfficialTM_3", "Sanchooo10", "CalvertLewin14", "masonmount_10", "PhilFoden",
                    "BenChilwell", "ben6white", "samjohnstone50", "reecejames_24", "BukayoSaka87", "BellinghamJude"]

for player in list_of_players:
    print(f"Extracting {player}...")
    joined_df[player] = joined_df['accounts_mentioned'].str.contains(f"'username': '{player}'").astype(bool)

Extracting JPickford1...
Extracting kylewalker2...
Extracting LukeShaw23...
Extracting _DeclanRice...
Extracting HarryMaguire93...
Extracting JackGrealish...
Extracting JHenderson...
Extracting HKane...
Extracting sterling7...
Extracting MarcusRashford...
Extracting trippier2...
Extracting deanhenderson...
Extracting Kalvinphillips...
Extracting OfficialTM_3...
Extracting Sanchooo10...
Extracting CalvertLewin14...
Extracting masonmount_10...
Extracting PhilFoden...
Extracting BenChilwell...
Extracting ben6white...
Extracting samjohnstone50...
Extracting reecejames_24...
Extracting BukayoSaka87...
Extracting BellinghamJude...


In [14]:
# Sort by ascending date
joined_df.sort_values('created_at', axis=0, inplace=True)

In [15]:
joined_df = joined_df[
    (joined_df["JPickford1"]) |
    (joined_df["kylewalker2"]) |
    (joined_df["LukeShaw23"]) |
    (joined_df["kylewalker2"]) |
    (joined_df["_DeclanRice"]) |
    (joined_df["HarryMaguire93"]) |
    (joined_df["JackGrealish"]) |
    (joined_df["JHenderson"]) |
    (joined_df["HKane"]) |
    (joined_df["sterling7"]) |
    (joined_df["MarcusRashford"]) |
    (joined_df["trippier2"]) |
    (joined_df["deanhenderson"]) |
    (joined_df["Kalvinphillips"]) |
    (joined_df["OfficialTM_3"]) |
    (joined_df["Sanchooo10"]) |
    (joined_df["CalvertLewin14"]) |
    (joined_df["masonmount_10"]) |
    (joined_df["PhilFoden"]) |
    (joined_df["BenChilwell"]) |
    (joined_df["ben6white"]) |
    (joined_df["samjohnstone50"]) |
    (joined_df["reecejames_24"]) |
    (joined_df["BukayoSaka87"]) |
    (joined_df["BellinghamJude"])
    ]

In [16]:
threshold_p = 0.5

joined_df['persp_over_threshold'] = (joined_df['identity_attack_score'] > threshold_p) | (joined_df['threat_score'] > threshold_p) | (joined_df['severe_toxicity_score'] > threshold_p)
                                                                                 

<ipython-input-16-96209b47e4e1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joined_df['persp_over_threshold'] = (joined_df['identity_attack_score'] > threshold_p) | (joined_df['threat_score'] > threshold_p) | (joined_df['severe_toxicity_score'] > threshold_p)


In [17]:
p_tweets = joined_df['persp_over_threshold'].sum()
total_tweets = len(joined_df)
print(f"Total tweets flagged by Perspective: {p_tweets}")
print(f"Total tweets: {total_tweets}")
print(f"Percentage of tweets flagged by Perspective: {(100*p_tweets)/total_tweets}%")

Total tweets flagged by Perspective: 1216
Total tweets: 90960
Percentage of tweets flagged by Perspective: 1.3368513632365875%


In [18]:
for player in list_of_players:
    joined_df[f'{player}_offensive_p'] = joined_df['persp_over_threshold'] & joined_df[player]

<ipython-input-18-bc99ad77bdcc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joined_df[f'{player}_offensive_p'] = joined_df['persp_over_threshold'] & joined_df[player]


In [20]:
player_offensive_tweet_map = pd.DataFrame(columns=["username",
                                                   "tweets_received",
                                                   "offensive_tweets_received_p",
                                                   "percentage_offensive_p"
                                                   ])

i = 0
for player in list_of_players:
    off_tweets_p = joined_df[player + '_offensive_p'].sum()
    tweets = joined_df[player].sum()
    percentage_p = 100 * (off_tweets_p / tweets)
    player_offensive_tweet_map.loc[i] = player, tweets, off_tweets_p, percentage_p
    i += 1

<ipython-input-20-512e9e64f32b>:11: RuntimeWarning: invalid value encountered in long_scalars
  percentage_p = 100 * (off_tweets_p / tweets)


In [22]:
name_map = {
    "JPickford1": "Jordan Pickford",
    "Popey1992": "Nick Pope",
    "kylewalker2": "Kyle Walker",
    "Kalvinphillips": "Kalvin Phillips",
    "ben6white": "Benjamin White",
    "ericdier": "Eric Dier",
    "trippier2": "Kieran Trippier",
    "TrentAA": "Trent Alexander-Arnold",
    "AaronRamsdale98": "Aaron Ramsdale",
    "JHenderson": "Jordan Henderson",
    "PhilFoden": "Phil Foden",
    "CallumWilson": "Callum Wilson",
    "sterling7": "Raheem Sterling",
    "LukeShaw23": "Luke Shaw",
    "_DeclanRice": "Declan Rice",
    "Madders10": "James Maddison",
    "BellinghamJude": "Jude Bellingham",
    "JackGrealish": "Jack Grealish",
    "HarryMaguire93": "Harry Maguire",
    "MarcusRashford": "Marcus Rashford",
    "BukayoSaka87": "Bukayo Saka",
    "HKane": "Harry Kane"
}

player_offensive_tweet_map["player"] = player_offensive_tweet_map["username"].map(name_map)

In [23]:
player_offensive_tweet_map.sort_values('tweets_received', inplace=True)

fig = go.Figure(data=[
    go.Bar(name='offensive', y=player_offensive_tweet_map.player, x=player_offensive_tweet_map.offensive_tweets_received_p, marker_color = "darkred", orientation = 'h'),
    go.Bar(name='total', y=player_offensive_tweet_map.player, x=player_offensive_tweet_map.tweets_received, marker_color="lightgrey", orientation = 'h')
])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.update_xaxes(type="log")

fig.show()

In [24]:
fig = go.Figure(data=[
    go.Bar(name='offensive', y=player_offensive_tweet_map.sort_values('offensive_tweets_received_p', axis=0, ascending=True, inplace=False).player, x=player_offensive_tweet_map.sort_values('offensive_tweets_received_p', axis=0, ascending=True, inplace=False).offensive_tweets_received_p, marker_color = "darkred", orientation = 'h')
])
# Change the bar mode

fig.show()

# Analysis

In [25]:
# Print some headline figures
print(f"Earliest tweet: {joined_df['created_at'].min()}")
print(f"Latest tweet: {joined_df['created_at'].max()}")

print(f"Number of tweets (player-only): {len(joined_df)}")

Earliest tweet: 2022-12-09 00:00:46+00:00
Latest tweet: 2022-12-15 23:59:07+00:00
Number of tweets (player-only): 90960


In [26]:
# After the final - approximate
a = joined_df[(joined_df['created_at'] > '2021-07-11 21:45:00+00:00')]
hateful_tweets = a['persp_over_threshold'].sum()
hateful_mentions = player_offensive_tweet_map['offensive_tweets_received_p'].sum() 
total_tweets = len(a)
print(f"Total tweets: {total_tweets}")
print(f"Hateful tweets: {hateful_tweets}")
print(f"Percenage: {100 * (hateful_tweets / total_tweets)}%")
print(f"Hateful mentions (i.e. not unique tweets): {hateful_mentions}")


Total tweets: 90960
Hateful tweets: 1216
Percenage: 1.3368513632365875%
Hateful mentions (i.e. not unique tweets): 1300


In [27]:
print(player_offensive_tweet_map[['username', 'offensive_tweets_received_p']].sort_values('offensive_tweets_received_p', ascending=False).head(10))

          username offensive_tweets_received_p
7            HKane                         895
9   MarcusRashford                          99
23  BellinghamJude                          61
4   HarryMaguire93                          60
22    BukayoSaka87                          58
3      _DeclanRice                          29
6       JHenderson                          15
17       PhilFoden                          14
1      kylewalker2                          13
5     JackGrealish                          12


In [28]:
# Percentage of hateful mentions that were directed at Saka, Sancho, Rashford
three_lions_perc = player_offensive_tweet_map[['offensive_tweets_received_p']].sort_values('offensive_tweets_received_p', ascending=False).head(3).sum() / hateful_mentions
print(f"{100 * three_lions_perc}%")


offensive_tweets_received_p    81.153846
dtype: object%


In [31]:
print('Five-minute intervals with most offensive tweets:')
print(joined_df.resample(pd.offsets.Minute(5), on='created_at')['persp_over_threshold'].sum().sort_values(ascending=False).head(5))
print('\n')
print('1 hour intervals with most offensive tweets:')
print(joined_df.resample(pd.offsets.Hour(1), on='created_at')['persp_over_threshold'].sum().sort_values(ascending=False).head(5))
print('\n')
print('24 hour intervals with most offensive tweets:')
print(joined_df.resample(pd.offsets.Day(1), on='created_at')['persp_over_threshold'].sum().sort_values(ascending=False).head(5))
print('\n')

flagged = joined_df['persp_over_threshold'].sum()
print(f"{flagged} offensive tweets, {100 * (flagged / len(joined_df))}% of all tweets aimed at players")

Five-minute intervals with most offensive tweets:
created_at
2022-12-10 21:00:00+00:00    82
2022-12-10 21:05:00+00:00    57
2022-12-10 20:45:00+00:00    41
2022-12-10 21:15:00+00:00    35
2022-12-10 21:10:00+00:00    29
Name: persp_over_threshold, dtype: int64


1 hour intervals with most offensive tweets:
created_at
2022-12-10 21:00:00+00:00    292
2022-12-10 20:00:00+00:00    100
2022-12-11 10:00:00+00:00     82
2022-12-11 11:00:00+00:00     77
2022-12-10 22:00:00+00:00     52
Name: persp_over_threshold, dtype: int64


24 hour intervals with most offensive tweets:
created_at
2022-12-11 00:00:00+00:00    561
2022-12-10 00:00:00+00:00    523
2022-12-12 00:00:00+00:00     90
2022-12-13 00:00:00+00:00     22
2022-12-14 00:00:00+00:00      9
Name: persp_over_threshold, dtype: int64


1216 offensive tweets, 1.3368513632365875% of all tweets aimed at players


# Visulisations - tweet frequency

In [32]:
# Setup vars for high-profile tweet times
kane_tweet_time = pd.to_datetime("2022-12-11T09:41:00.000Z")

full_time = pd.to_datetime("2022-12-10T20:55:00.000Z")

kick_off = pd.to_datetime("2022-12-10T19:00:00.000Z")

In [33]:
INTERVAL = pd.offsets.Minute(5)

In [34]:
def create_frequency_plotly(df, y_values, y_value_names, title, nticks):
    fig = go.Figure()
    i = 0
    for i in range(len(y_values)):
        fig.add_trace(go.Scatter(x=df['created_at'], y=df[y_values[i]],
                      mode='lines',
                      name=y_value_names[i]))
        
    fig.update_layout(
        xaxis_title="Time",
        yaxis_title="Number of tweets",
        xaxis = {
          'tickformat': '%d %B',
          'tickmode': 'auto',
          'nticks': nticks,
        },
        xaxis_tickformat = '%d %B',
        title= {
          'text': title,
          'y':0.9,
          'x':0.5,
          'xanchor': 'center',
          'yanchor': 'top'
        }
    )
    return fig

def create_frequency_bars(df, y_values, title, nticks):
    fig = go.Figure()
    for y_value in y_values:
        fig.add_trace(go.Bar(x=df['created_at'], y=df[y_value],
                      name=y_value))
    fig.update_layout(
        xaxis_title="Date",
        yaxis_title="Number of tweets",
        xaxis = {
          'tickformat': '%d %B',
          'tickmode': 'auto',
          'nticks': nticks,
        },
        xaxis_tickformat = '%d %B',
        title= {
          'text': title,
          'y':0.9,
          'x':0.5,
          'xanchor': 'center',
          'yanchor': 'top'
        }
    )

    return fig

In [41]:
# Plot all tweets frequency
df = joined_df[(joined_df['created_at'] > '2022-12-10 13:00:00+00:00') & (joined_df['created_at'] < '2022-12-13 00:00:00+00:00')].resample(pd.offsets.Minute(15), on='created_at')['tweet_id'].count().reset_index()
fig = create_frequency_bars(df, ['tweet_id'], "", 4)

y_max = max(df.iloc[(df['created_at']-full_time).abs().argsort()[:10]]['tweet_id'].tolist())
fig.add_annotation(x=kick_off, y=300,
    text="Kick-off",
    showarrow=True,
    arrowhead=1)
fig.add_annotation(x=full_time, y=y_max + (y_max / 100),
    text="End of game",
    showarrow=True,
    arrowhead=1)
fig.add_annotation(x=kane_tweet_time, y=max(df.iloc[(df['created_at']-kane_tweet_time).abs().argsort()[:10]]['tweet_id'].tolist()) + (y_max / 100),
    text="Kane tweet",
    showarrow=True,
    arrowhead=1)
fig.write_html("/content/drive/MyDrive/University/Dissertation/images/qf_freq.html", include_plotlyjs='cdn')
fig.show()

In [42]:
# Plot Kane tweet frequency
df = joined_df[(joined_df['created_at'] > '2022-12-10 13:00:00+00:00') & (joined_df['created_at'] < '2022-12-13 00:00:00+00:00')].resample(INTERVAL, on='created_at')['HKane'].sum().reset_index()
y_values = ['HKane']
fig = create_frequency_plotly(df, y_values, y_values, "", 6)

y_max = max(df.iloc[(df['created_at']-full_time).abs().argsort()[:10]]['HKane'].tolist())
fig.add_annotation(x=kick_off, y=max(df.iloc[(df['created_at']-kick_off).abs().argsort()[:10]]['HKane'].tolist()) + (y_max / 100),
    text="Kick-off",
    showarrow=True,
    arrowhead=1)
fig.add_annotation(x=full_time, y=y_max + (y_max / 100),
    text="End of game",
    showarrow=True,
    arrowhead=1)
fig.add_annotation(x=kane_tweet_time, y=max(df.iloc[(df['created_at']-kane_tweet_time).abs().argsort()[:10]]['HKane'].tolist()) + (y_max / 100),
    text="Kane tweet",
    showarrow=True,
    arrowhead=1)

fig.show()

In [48]:
# Plot Kane tweet frequency
df = joined_df[(joined_df['created_at'] > '2022-12-10 13:00:00+00:00') & (joined_df['created_at'] < '2022-12-13 00:00:00+00:00')].resample(INTERVAL, on='created_at')['HKane_offensive_p'].sum().reset_index()
y_values = ['HKane_offensive_p']
fig = create_frequency_plotly(df, y_values, y_values, "", 6)

y_max = max(df.iloc[(df['created_at']-full_time).abs().argsort()[:10]]['HKane_offensive_p'].tolist())
fig.add_annotation(x=kick_off, y=max(df.iloc[(df['created_at']-kick_off).abs().argsort()[:10]]['HKane_offensive_p'].tolist()) + (y_max / 100),
    text="Kick-off",
    showarrow=True,
    arrowhead=1)
fig.add_annotation(x=full_time, y=y_max + (y_max / 100),
    text="End of game",
    showarrow=True,
    arrowhead=1)
fig.add_annotation(x=kane_tweet_time, y=max(df.iloc[(df['created_at']-kane_tweet_time).abs().argsort()[:10]]['HKane_offensive_p'].tolist()) + (y_max / 100),
    text="Kane tweet",
    showarrow=True,
    arrowhead=1)

fig.show()